### Chapter 2
#### MNIST - CNN

In [ ]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# deprecatioin warnings
mnist_data = input_data.read_data_sets("MNIST_data", one_hot=True)

### Placeholders

In [ ]:
input_size = 784  # flattened images 28x28 as a vector
no_classes = 10
batch_size = 100
total_batches = 200

In [ ]:
x_input = tf.placeholder(tf.float32, shape=[None, input_size])
y_input = tf.placeholder(tf.float32, shape=[None, no_classes])

### TensorBoard
#### Summary Writer

In [ ]:
def add_variable_summary(tf_variable, summary_name):
    with tf.name_scope(summary_name + '_summary'):
        # calc mean
        mean = tf.reduce_mean(tf_variable)
        tf.summary.scalar('Mean', mean)
        # calc std dev
        with tf.name_scope('standard_deviation'):
            standard_deviation = tf.sqrt(tf.reduce_mean(tf.square(tf_variable - mean)))
            
        tf.summary.scalar('StandardDeviation', standard_deviation)
        tf.summary.scalar('Maximum', tf.reduce_max(tf_variable))
        tf.summary.scalar('Minimum', tf.reduce_min(tf_variable))
        tf.summary.histogram('Histogram', tf_variable)

### Input Data
shape each MNIST digit as 28x28x1

In [ ]:
# reshape operation
x_input_reshape = tf.reshape(x_input, [-1,28,28,1], name='input_reshape')

### Convolution Layer Function
return a convolution layer  
include a tensorboard summary

In [ ]:
def convolution_layer(input_layer, filters, kernel_size=[3,3], activation=tf.nn.relu):
    layer = tf.layers.conv2d(
        inputs=input_layer,
        filters=filters,
        kernel_size=kernel_size,
        activation=activation)
    add_variable_summary(layer, 'convolution')
    return layer

### Pooling Layer Function
return a pooling layer  
include a tensorboard summary

In [ ]:
def pooling_layer(input_layer, pool_size=[2,2], strides=2):
    layer = tf.layers.max_pooling2d(
        inputs=input_layer,
        pool_size=pool_size,
        strides=strides)
    add_variable_summary(layer, 'pooling')
    return layer

### Dense Layer Function
input - a single dimension (flat) vector  
 a hidden layer (units), 1024 in this example  
output reLU activation   
return a dense layer  

include a tensorboard summary

In [ ]:
def dense_layer(input_layer,units, activation=tf.nn.relu):
    layer = tf.layers.dense(
        inputs=input_layer,
        units=units,
        activation=activation)
    add_variable_summary(layer, 'dense')
    return layer

### Assemble the CNN

In [ ]:
# layer 1
convolution_layer_1 = convolution_layer(x_input_reshape, 64)   # 64 filters
pooling_layer_1 = pooling_layer(convolution_layer_1)

# layer 2
convolution_layer_2 = convolution_layer(pooling_layer_1, 128)  # 128 filters
pooling_layer_2 = pooling_layer(convolution_layer_2)

flattened_pool = tf.reshape(pooling_layer_2, [-1, 5*5*128], name = 'flattened_pool')    # n, 5x5, filters

dense_layer_bottleneck = dense_layer(flattened_pool, 1024)

##### Drop Out
layer 2 -> boolean x Dropout -> Dense

In [ ]:
dropout_bool = tf.placeholder(tf.bool)  # placeholder
dropout_layer = tf.layers.dropout(
    inputs=dense_layer_bottleneck,
    rate=0.4,
    training=dropout_bool)

In [ ]:
logits = dense_layer(dropout_layer, no_classes)

### Loss

In [ ]:
with tf.name_scope('loss'):
    softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
        labels=y_input, logits=logits)
    loss_operation = tf.reduce_mean(softmax_cross_entropy, name='loss')
    tf.summary.scalar('loss', loss_operation)

### Training Optimizer Function

In [ ]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer().minimize(loss_operation)

### Model Accuracy

In [ ]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        predictions = tf.argmax(logits, 1)
        correct_predictions = tf.equal(predictions, tf.argmax(y_input, 1))
    with tf.name_scope('accuracy'):
        accuracy_operation = tf.reduce_mean(
            tf.cast(correct_predictions, tf.float32))
        
tf.summary.scalar('accuracy', accuracy_operation)

### Session & Initialize Variables

In [ ]:
session = tf.Session()
session.run(tf.global_variables_initializer())

### Summary Writer
the graph is written once  

In [ ]:
merged_summary_operation = tf.summary.merge_all()
train_summary_writer = tf.summary.FileWriter('/tmp/train', session.graph)
test_summary_writer  = tf.summary.FileWriter('/tmp/test')

### Training Loop


In [ ]:
test_images, test_labels = mnist_data.test.images, mnist_data.test.labels

In [ ]:

for batch_no in range(total_batches):
    # retreive a batch
    mnist_batch = mnist_data.train.next_batch(batch_size)
    train_images, train_labels = mnist_batch[0], mnist_batch[1]
    if batch_no == 0:
        print ("train_Images", train_images.shape)
        print ("train_labels", train_labels.shape)
    print ("batch#", batch_no)
    # run the batch through the model
    #   merged_summary is the outut values of the optimizer
    _, merged_summary = session.run([optimizer, merged_summary_operation], feed_dict={
        x_input: train_images,
        y_input: train_labels,
        dropout_bool: True
    })
    # summary_writer
    train_summary_writer.add_summary(merged_summary, batch_no)
    # add test statistics every 10th loop
    if batch_no % 10 == 0:
        merged_summary, _ = session.run([merged_summary_operation, accuracy_operation], feed_dict={
            x_input: test_images,
            y_input: test_labels,
            dropout_bool: False
        })
        test_summary_writer.add_summary(merged_summary, batch_no)
    